In [ ]:
# HTTP lib
import requests

# HTML/XML parsing lib
from bs4 import BeautifulSoup

# Dataframe lib
import pandas

# Methods for parsing 10-Q and 10-K filings 
Given a root filing directory ***with*** trailing slash (https://www.sec.gov/Archives/edgar/data/CIK#/ACCESSION#/) we first look for FilingSummary.xml and go from there, in the case of modern filings (with XBRL data). These modern 10-Q and 10-K filings hold their financial reports in identical formats thus the same methods should be applicable to both.

*Note that filings without XBRL data not have FilingSummary.xml nor a primary document listed in the JSON data returned from a CIK submission search. Rather the "primary document" of the filing can be found at [Archives/edgar/data/CIK/ACCESSION_WITH_DASHES.txt](https://). Separate methods will be called to parse the filing in this case.*



This first method simply confirms the existence of FilingSummary.xml by crawling through the index.json file. It returns the path to the filing summary if it exists, and and empty string if not.

In [ ]:
# NOTE: INCLUDE TRAILING SLASH WHEN CALLING
def confirm_modern_filing_summary(root_filing_dir):

  # Need to set user agent for sec.gov
  request_headers = { "User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36" }

  # Request root_filing_dir/index.json
  index_path = r"{}index.json".format(root_filing_dir)
  response = requests.get(url = index_path, headers = request_headers)
  response.raise_for_status()

  index_json = response.json()
  
  # directory->item is a list of dictionaries, one per each file in the directory
  # Make sure that the "name" key of one of these dictionaries is "FilingSummary.xml"
  try:
    for current_item in index_json['directory']['item']:
      if current_item['name'] == "FilingSummary.xml":
        return root_filing_dir + "FilingSummary.xml"

  except:
    pass
  return ""

# Parsing the FilingSummary.xml
This file contains, under section MyReports, multiple Report instances structured as such:



```
<Report instance="abcl-10q_20220630.htm">
  <IsDefault>false</IsDefault>
  <HasEmbeddedReports>false</HasEmbeddedReports>
  <HtmlFileName>R1.htm</HtmlFileName>
  <LongName>100000 - Document - Document and Entity Information</LongName>
  <ReportType>Sheet</ReportType>
  <Role>http://www.abcellera.com/20220630/taxonomy/role/DocumentDocumentAndEntityInformation</Role>
  <ShortName>Document and Entity Information</ShortName>
  <MenuCategory>Cover</MenuCategory>
  <Position>1</Position>
</Report>
```
The MenuCategory field is somewhat revealing as well the short name. MenuCategory is set to "Statements" for financial reports, "Notes" for remarks/disclosures etc (see 10q_k_notes_parsing). Each report lives at R{Position}.htm. This is true of both modern 10-Q and 10-K filings.

A few support methods for organize_modern_financial_statements method below:

In [ ]:
# Returns True if the given shortname is determined to be a balance sheet
def is_balance_sheet_from_shortname(target_shortname):
  
  if 'balance sheet' in target_shortname.lower():
    return True
  if 'financial position' in target_shortname.lower():
    return True

  return False

In [ ]:
# Returns True if the given shortname is determined to be an income statement
def is_income_statement_from_shortname(target_shortname):

  if 'of income' in target_shortname.lower():
    return True
  if 'of operations' in target_shortname.lower():
    return True

  return False

In [ ]:
# Returns True if the given shortname is determined to be a statement of cash flows
def is_cashflow_statement_from_shortname(target_shortname):

  if 'cash flow' in target_shortname.lower():
    return True
  return False

In [ ]:
# Returns True if the given shortname is determined to be a statement of owners equity
def is_oe_statement_from_shortname(target_shortname):

  if 'shareholders' in target_shortname.lower():
    return True
  if 'stakeholders' in target_shortname.lower():
    return True
  if 'stockholders' in target_shortname.lower():
    return True
  if 'owner\'s' in target_shortname.lower():
    return True
  if 'of equity' in target_shortname.lower():
    return True
  if 'in equity' in target_shortname.lower():
    return True

  return False

The below function locates individual financial statements from a filing, given the path to FilingSummary.xml (hence the modern keyword in the method name). A dictionary with the following structure is returned:


```
{
  "SHORTNAME" : "URL",
  ...
}

```


In [47]:
def find_modern_financial_reports(filing_summary_path):

  # Base filing directory path
  base_filing_path = filing_summary_path.replace('FilingSummary.xml', '')

  # Dictionary structure we will return
  statements_dictionary = {}

  # GET FilingSummary.xml
  request_headers = {"User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  response = requests.get(url = filing_summary_path, headers = request_headers)
  response.raise_for_status()

  xml_content = response.content
  soup_summary = BeautifulSoup(xml_content, "lxml")

  # Find MyReports and loop through them
  reports = soup_summary.find('myreports')
  for current_report in reports.find_all('report'):

    # The last report will not have some tags such as MenuCategory and will cause an exception trying to read it, thus try + except
    try:
      if current_report.menucategory.text.lower() == "statements":
        
        # Add to return dict
        statements_dictionary[current_report.shortname.text.strip()] = base_filing_path + current_report.htmlfilename.text.strip()

    # Expected to be hit, no problem    
    except:
      pass

  return statements_dictionary

In [45]:
"""
Takes a dictionary of statements returned by the above method, and organizes them into the 4 main financial statement categories. Returns a dict of structure (note dictionary values are lists):
{
  'balance_sheet' : [ 'URL1', ...],
  'income_statement' : [],
  'cashflow_statement' : [],
  'oe_statement' : []
}
"""
def organize_modern_financial_reports(statements_dict):

  # Returned structure
  main_statements = {
  'balance_sheet' : [],
  'income_statement' : [],
  'cashflow_statement' : [],
  'oe_statement' : []
  }
  
  # Loop through statements passed
  for key, val in statements_dict.items():

    # Check the shortname against regex candidates for each of the 4 statements
    if is_balance_sheet_from_shortname(key):
      main_statements["balance_sheet"].append(val)
    elif is_income_statement_from_shortname(key):
      main_statements["income_statement"].append(val)
    elif is_cashflow_statement_from_shortname(key):
      main_statements["cashflow_statement"].append(val)
    elif is_oe_statement_from_shortname(key):
      main_statements["oe_statement"].append(val)
    else:
      pass
    
  return main_statements

In [49]:
# POC
modern_filing_summary = confirm_modern_filing_summary("https://www.sec.gov/Archives/edgar/data/0001703057/000156459022028784/")
if modern_filing_summary:
  statements_dict = find_modern_financial_reports(modern_filing_summary)
  if statements_dict:
    organized_statements = organize_modern_financial_reports(statements_dict)
    print(organized_statements)


{'balance_sheet': ['https://www.sec.gov/Archives/edgar/data/0001703057/000156459022028784/R2.htm', 'https://www.sec.gov/Archives/edgar/data/0001703057/000156459022028784/R3.htm'], 'income_statement': ['https://www.sec.gov/Archives/edgar/data/0001703057/000156459022028784/R4.htm'], 'cashflow_statement': ['https://www.sec.gov/Archives/edgar/data/0001703057/000156459022028784/R6.htm'], 'oe_statement': ['https://www.sec.gov/Archives/edgar/data/0001703057/000156459022028784/R5.htm']}


# Parsing the Statements
After locating the four main financial statements, we move on the parsing them. As with above, parsing methods are for the most part divided between "modern" filings and "legacy" filings and named accordingly. 

Basically the legacy filings contain the entire filing in full text at the single HTML document [Archives/edgar/data/CIK/ACCESSION_WITH_DASHES.txt](https://), rather than each report/statement being its own document as in the modern filings. More information below.

# Parsing Modern Statements
For modern filings, each statement/report lives on its own .htm document and its data is contained in a table object with the following format:


```
<table class="report" border="0" cellspacing="2" id="idmXXXXXXXXXXXXXXX">
  <tbody>
    <tr>... table headers row (basically defines table columns) ...</tr>
      <th class=...> one "th" object per column </th>
    <tr class="rXX">... either data / section header row...</tr>
      <td class=...> one "td" object per column in each row </td>
    ... data rows
  </tbody>
</table>
```
The table headers are easily identified by the `<th>` tag, but section headers (labels grouping multiple actual entries together such as "Current Assets") are most easily identified by the use of `<strong>` tags to bolden them on the table, along with their lack of `<th>` tags *. Actual rows of data values will have neither of these two attributes.  

*Some section header style rows, mostly encountered in income statements but also companies with subsidiaries, do not use `<strong>` tags, but rather seem to have a `<class>` of "rh". The class tag seems to denote the shading/style of the row. Whether it is a total/sum, etc. These heavy "rh" rows seem to denote different sections such as different "members"/categories/subsidiaries of the company. The rows up until the first "rh" are aggregated / totals for the whole company. Methods are supplied below for breaking down reports into sub-tables according to "rh" rows. 


**A note on table headers**: in many income statements, such as example [here](https://www.sec.gov/Archives/edgar/data/1703057/000156459022028784/R4.htm), there are two lines of table headers (and even more could be possible I suppose). From my own inspection and looking around, it seems standard for companies to list income/expenses for the 3 months ended on a certain date and 6 months ended on a certain date. read_modern_report_tables will in turn append a new list to report_data['headers'] for each header line/row it encounters. For now, in regards to income statements, we will assume the columns are ordered as follows (again see above example report):
```
'3 Months Ended ...(current year)' | '3 Months Ended ...(year ago)' | '6 Months Ended ...(current year)' | '6 Months Ended ...(year ago)'
```
Thus we really just want the dates, which are in the second list of report_data['headers'] if it exists. 

In [ ]:
def read_modern_report_tables(report_document_url):

  request_headers = { "User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36" }

  # Dictionary we will return
  report_data = {}
  report_data['headers'] = []
  report_data['sections'] = []
  report_data['data'] = []

  # Request the document contents
  response = requests.get(url = report_document_url, headers = request_headers)
  response.raise_for_status()

  html_content = response.content
  report_soup = BeautifulSoup(html_content, 'lxml')

  # In case there are multiple tables in the document, loop through all of those labeled with the "report" class. Tables of other classes (i.e. of type "authRefData") are ignored.
  for table_index, current_table in enumerate(report_soup.find_all('table', class_ = "report")):

    # Loop through table's rows 
    for row_index, current_row in enumerate(current_table.find_all('tr')):

      # If we come across a row of class "rh" (signaling an end of the aggregated/totals and beginning of member/source breakdowns), end here tell user they may want to further split the table 
      # So far this structure seems to hold true as discussed in the "*" above. 
      try:
        if "rh" in current_row['class']:
          print("Member/source-specific financial information is available but will not be recorded here. Consider further splitting the report by member.")
          break
      except:
        pass

      # Grab all the elements / columns of the row. Keep in mind column headers don't have <td> tags, thus we list <th> objects within the elif upon finding them
      line_columns = current_row.find_all('td')

      # Decide if row is: data, section (sub) header, or column header according to logic above (th and strong tags) and append it to the proper list (headers, sections, or data) of the statement_data dictionary.

      # Data row. May be a footnote or superscript, which we will filter out
      if (len(current_row.find_all('th')) == 0 and len(current_row.find_all('strong')) == 0):
        data_row = [] 
      
        try:
          for i in current_row['class']: # We need to do a sub-string search, TODO optimize this loop
            # Skip rows containing "note" in a class tag (usually are footnotes)
            if "note" in i.lower(): 
              pass

            # Skip superscripted values/columns (usually link to footnote) and those with class of "fn", again link to footnote
            for current_column in line_columns:
              if len(current_column.find_all('sup')) == 0:
                try:
                  if "fn" not in current_column['class']:
                    data_column = current_column.text.strip()
                    data_row.append(data_column)
                except: # No column class set. Shouldn't happen but just ignore and try next column of the row
                  pass

            report_data['data'].append(data_row)

        # Most likely did not have a row class set. Generally means blank / separator
        except:
          pass

      # Section/sub header row
      elif (len(current_row.find_all('th')) == 0 and len(current_row.find_all('strong')) != 0):
        section_row = line_columns[0].text.strip() # Only the first element in this row will have the section label, the others are blank so no point
        report_data['sections'].append(section_row)

      # Header row
      elif len(current_row.find_all('th')) != 0:
        header_row = [ele.text.strip() for ele in current_row.find_all('th')]
        report_data['headers'].append(header_row)

      # Unable to identify
      else:
        print(r"Unable to identify row #{} of table #{} found in table of report: {}".format(row_index + 1, table_index + 1, report_document_url))

  # Remove any newline characters in the columns or section headers
  
  # Return the filled dictionary
  return report_data

In [ ]:

target_summary = "https://www.sec.gov/Archives/edgar/data/886093/000168316822006728/R3.htm"
rep_data = read_modern_report_tables(target_summary)
print(rep_data)

Member/source-specific financial information is available but will not be recorded here. Consider further splitting the report by member.
{'headers': [['Consolidated Balance Sheets (Unaudited) (Parenthetical) - $ / shares', 'Jun. 30, 2022', 'Dec. 31, 2021']], 'sections': [], 'data': []}


The below method, given the URL to a specific filing report obtained by parsing FilingSummary.xml, reads the report data neatly into a pandas dataframe. A dictionary is returned in the following format, passing the title of a report (as read from the header rows) along with the dataframe:


```
report_df_dict = {
  "title" : "Report Title",
  "df" : DATAFRAME
}
```



In [ ]:
def read_modern_report_into_df(report_document_url):
  
  # Read the data into a dictionary with 3 keys: headers, sections, and data
  report_data_dict = read_modern_report_tables(report_document_url)

  # Create two lists: headers (column headings), and data values. Ignore section headers for now
  report_headers_list = report_data_dict['headers']
  report_data_list = report_data_dict['data']

  # Create our return structure and get the report title
  report_df_dict = {
      "title" : "Unknown",
      "df" : None
  }
  try:
    report_df_dict["title"] = report_headers_list[0][0] # Should be the first element of the first column header row
  except: 
    pass

  # Check that data has been passed. Can be empty if the report starts off with an "rh" row, usually seen in parenthetical statements regarding member entities etc.
  if len(report_data_list) == 0:
    return report_df_dict

  # Create the dataframe around the report data list
  report_df = pandas.DataFrame(report_data_list)

  # Re-index the dataframe (first value/column of each sublist is the name of the financial item, i.e "Accounts Receivable")
  report_df.index = report_df[0]
  report_df.index.name = 'item/account'
  report_df = report_df.drop(0, axis = 1)

  # Sanitize it of illegal characters
  report_df = report_df.replace('[\[\]\$,)]', '', regex = True)\
  .replace('[(]', '-', regex = True)\
  .replace('', 'NaN', regex = True)

  # Convert data values to floats. "Unlimited" and other text may be present, so ignore for now. Could convert them to NaNs also
  report_df = report_df.astype(dtype = float, errors = 'ignore')

  # Drop rows with all NaN's
  #report_df = report_df.dropna(how="all")
  
  # Set column names to the headers we stored. Remember we have a list of lists. Do some cleaning
  # If there is only one list/row of column headers, we want to drop the first element (which basically holds the table name). Otherwise rely on the last row to be the dates / headings we want.
  # TD-DO: Better optimization for multi-line headers etc, also integrate section headers 

  if len(report_headers_list) == 1:
    report_df = report_df.set_axis(report_headers_list[0][1:], axis = 'columns')

  elif len(report_headers_list) > 1:
    report_df = report_df.set_axis(report_headers_list[-1], axis = 'columns')

  else:
    print("Failed to read/set column headers for dataframe of {}".format(report_document_url))

  # Copy the dataframe into our return structure
  report_df_dict["df"] = report_df
  return report_df_dict

In [ ]:
# POC
modern_filing_summary = confirm_modern_filing_summary("https://www.sec.gov/Archives/edgar/data/0001703057/000156459022028784/")
if modern_filing_summary:

  statements_dict = find_modern_financial_reports(modern_filing_summary)
  organized_statements = organize_modern_financial_reports(statements_dict)

  for report_type in organized_statements.keys():
    for report_url in organized_statements[report_type]:

      report_df_dict = read_modern_report_into_df(report_url)

      print("Report URL: {}".format(report_url))
      print("Report title: {}".format(report_df_dict["title"]))
      display(report_df_dict["df"])

In [ ]:
# TODO: Code to split reports by members

# Parsing Legacy Statements
As mentioned above, older filings contain the entire filing at one document given by [Archives/edgar/data/CIK/ACCESSION_WITH_DASHES.txt](https://). These full text documents contain a structure such as the following from a 1997 Amazon 10-Q (https://www.sec.gov/Archives/edgar/data/1018724/0000891020-97-001148.txt):

```
-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 EA8wP4EJ1aofn3wNjHXvYqpXzR6GMf4Bz9Y70HsR2k133p7FODrUtk4OmN9dGSoW
 Tmhu+MdHUHbxrFPB5Qw1Xw==

<SEC-DOCUMENT>0000891020-97-001148.txt : 19970815
<SEC-HEADER>0000891020-97-001148.hdr.sgml : 19970815
ACCESSION NUMBER:		0000891020-97-001148
CONFORMED SUBMISSION TYPE:	10-Q
PUBLIC DOCUMENT COUNT:		3
CONFORMED PERIOD OF REPORT:	19970630
FILED AS OF DATE:		19970814
SROS:			NASD

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AMAZON COM INC
		CENTRAL INDEX KEY:			0001018724
		STANDARD INDUSTRIAL CLASSIFICATION:	BOOKS: PUBLISHING OR PUBLISHING AND PRINTING [2731]
		IRS NUMBER:				911646860
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		10-Q
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	000-22513
		FILM NUMBER:		97661331

	BUSINESS ADDRESS:	
		STREET 1:		1516 SECOND AVE 4TH FLOOR
		STREET 2:		PO BOX 80387
		CITY:			SEATTLE
		STATE:			WA
		ZIP:			98101
		BUSINESS PHONE:		2066222335

	MAIL ADDRESS:	
		STREET 1:		PO BOX 80387
		STREET 2:		1516 SECOND AVE 4TH FLOOR
		CITY:			SEATTLE
		STATE:			WA
		ZIP:			98101
</SEC-HEADER>
<DOCUMENT>
<TYPE>10-Q
<SEQUENCE>1
<DESCRIPTION>EDGAR FORM 10-Q FOR AMAZON.COM, INC
<TEXT>
DOC TEXT
</TEXT>
</DOCUMENT>
<DOCUMENT>
ONE PER INDIVIDUAL DOCUMENT IN THE FILING
</DOCUMENT>
</SEC-DOCUMENT>
-----END PRIVACY-ENHANCED MESSAGE-----
```
In fact, there are two types of "legacy" filings. Newer "legacy" filings have HTML content inbetween `<TEXT>` tags. Older "legacy" filings are not HTML, just straight plain text. The methods used in parsing these legacy filings are named using the keyword "legacy" to differentiate them. Differentiating between the two legacy types, "html" and "plaintext" will be used in method names.